In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
import random
from collections import deque

def create_q_network(input_shape, action_space):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Conv2D(32, 8, strides=4, activation='relu'),
        layers.Conv2D(64, 4, strides=2, activation='relu'),
        layers.Conv2D(64, 3, strides=1, activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(action_space, activation='tanh')  # Activación para acciones continuas
    ])
    return model

def train(episodes):
    env = gym.make("CarRacing-v3", render_mode='human')  # Usamos render_mode='human' para mostrar el entorno
    print("Entorno 'CarRacing-v3' cargado.")  # Confirmación de que el entorno se ha cargado correctamente

    input_shape = env.observation_space.shape
    action_space = env.action_space.shape[0]  # Número de acciones continuas

    q_network = create_q_network(input_shape, action_space)
    target_network = create_q_network(input_shape, action_space)
    target_network.set_weights(q_network.get_weights())  # Inicializar con los mismos pesos

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    gamma = 0.99  # Factor de descuento
    epsilon = 1.0
    epsilon_min = 0.1
    epsilon_decay = 0.995
    batch_size = 64
    memory = deque(maxlen=100000)  # Memoria para experiencia
    reward_per_episode = np.zeros(episodes)

    for episode in range(episodes):
        state, _ = env.reset()
        state = np.array(state, dtype=np.float32)  # Convertir el estado a un tipo adecuado
        episode_reward = 0
        done = False
        step_count = 0  # Contador de pasos para medir el progreso

        print(f"Comenzando episodio {episode + 1}")  # Confirmación de inicio de episodio

        # Establecer el umbral de recompensa para completar el 50% de la pista
        max_steps = 1000  # Establecer un número máximo de pasos para todo el episodio
        progress_threshold = 0.40  # 50% de progreso

        while not done:
            if random.random() < epsilon:
                action = env.action_space.sample()  # Exploración
            else:
                action = q_network.predict(np.expand_dims(state, axis=0)).flatten()  # Explotación

            # Ejecutar la acción y obtener el siguiente estado
            next_state, reward, done, _, _ = env.step(action)
            next_state = np.array(next_state, dtype=np.float32)

            # Acumular la recompensa
            episode_reward += reward
            step_count += 1

            # Si el número de pasos supera el 50% de los pasos máximos, finalizar el episodio
            if step_count > (progress_threshold * max_steps):
                done = True
                print(f"Episodio {episode + 1} terminado porque el agente completó el 50% del recorrido. Recompensa: {episode_reward}")

            # Si la recompensa es muy negativa (el coche salió de la pista), finalizar el episodio
            if reward < -50:  # Ajusta este valor si es necesario
                done = True
                print(f"Episodio {episode + 1} terminado porque el coche salió de la pista. Recompensa: {reward}")

            # Almacenar la transición en la memoria
            memory.append((state, action, reward, next_state, done))

            # Actualizar el estado actual
            state = next_state

            # Entrenamiento de la red neuronal
            if len(memory) > batch_size:
                minibatch = random.sample(memory, batch_size)
                states, actions, rewards, next_states, dones = zip(*minibatch)

                states = np.array(states)
                next_states = np.array(next_states)

                # Predicción de la Q-Value actual
                q_values = q_network.predict(states)

                # Predicción de la Q-Value objetivo
                q_values_next = target_network.predict(next_states)

                # Actualizar la Q-Value de la acción tomada
                for i in range(batch_size):
                    q_values[i] = rewards[i] + gamma * np.max(q_values_next[i]) * (1 - dones[i])

                # Entrenamiento de la red
                with tf.GradientTape() as tape:
                    q_values_pred = q_network(states)
                    loss = tf.reduce_mean(tf.square(q_values_pred - q_values))
                grads = tape.gradient(loss, q_network.trainable_variables)
                optimizer.apply_gradients(zip(grads, q_network.trainable_variables))

        # Reducir epsilon
        epsilon = max(epsilon * epsilon_decay, epsilon_min)

        # Mostrar progreso cada 100 episodios
        reward_per_episode[episode] = episode_reward

        if (episode + 1) % 100 == 0:
            print(f"Episodio: {episode + 1} - Recompensa acumulada: {reward_per_episode[episode]}")

        # Actualizar la red objetivo cada 10 episodios
        if (episode + 1) % 10 == 0:
            target_network.set_weights(q_network.get_weights())

        # Forzar el reinicio del entorno cuando el episodio se termine
        if done:
            state, _ = env.reset()  # Reiniciar el entorno después de que el episodio termine

    # Graficar la recompensa acumulada por episodio
    plt.plot(reward_per_episode)
    plt.xlabel("Episodios")
    plt.ylabel("Recompensa acumulada")
    plt.title("Desempeño del agente durante el entrenamiento")
    plt.show()

    # Evaluación del agente entrenado DESPUES DEL EPISODIO 1000
    if episodes >= 1000:  # Solo hacer evaluación después de 1000 episodios
        print(f"Evaluación después del episodio 1000")
        state, _ = env.reset()
        state = np.array(state, dtype=np.float32)
        done = False

        # Evaluación con renderizado visual
        while not done:
            action = q_network.predict(np.expand_dims(state, axis=0)).flatten()
            state, reward, done, _, _ = env.step(action)
            state = np.array(state, dtype=np.float32)
            env.render()  # Mostrar la visualización gráfica del entorno

        env.close()  # Cerrar el entorno al finalizar

# Ejecutar el entrenamiento con 10000 episodios para una prueba rápida
train(100)  # Cambié a 10000 episodios para mayor entrenamiento